In [1]:
from flask import Flask, render_template
from flask_cors import CORS

import json
import os
import time

In [2]:
app = Flask(__name__)

CORS(app)

In [3]:
@app.route("/")
def index():
    return render_template('index.html')


@app.route("/string/<word>/<stime>/<etime>", methods=["GET"])
def string_tweet(word, stime, etime):
    start_time = time.time()
    inCache, res = get_from_cache('string', word)
    if inCache:
        print("time taken:", time.time() - start_time)
        return json.dumps(time_filter(res, stime, etime, time.time() - start_time))
    
    res = getTweetsbyString(word)
    
    add_to_cache('string', word, res)
    print("time taken:", time.time() - start_time)
    return json.dumps(time_filter(res, stime, etime, time.time() - start_time))

@app.route("/hashtag/<word>/<stime>/<etime>", methods=["GET"])
def hashtag_tweet(word, stime, etime):
    start_time = time.time()
    inCache, res = get_from_cache('hashtag', word)
    if inCache:
        return json.dumps(time_filter(res, stime, etime, time.time() - start_time))
        
    res = getTweetsbyHashtag(word)
    
    add_to_cache('hashtag', word, res)
    return json.dumps(time_filter(res, stime, etime, time.time() - start_time))

@app.route("/user/<word>/<stime>/<etime>", methods=["GET"])
def user_tweet(word, stime, etime):
    start_time = time.time()
    inCache, res = get_from_cache('user', word)
    if inCache:
        return json.dumps(time_filter(res, stime, etime, time.time() - start_time))
        
    usr_lst = getUserByWord(word)
    id_lst = extract_val('id', usr_lst)
    res = getTweetsbyUserIDsList(id_lst)
    
    add_to_cache('user', word, res)
    return json.dumps(time_filter(res, stime, etime, time.time() - start_time))


@app.route("/author/<id>", methods=["GET"])
def get_author(id):
    start_time = time.time()
    inCache, res = get_from_cache('author', id)
    if inCache:
        return json.dumps({'data': res, 'fetch_time': time.time() - start_time })
    
    res = getUserByID(id)
    res['tweets'] = getTweetsbyUserID(id)
    res['retweets'] = getRetweetsbyUserID(id)
    
    add_to_cache('author', id, res)
    return json.dumps({'data': res, 'fetch_time': time.time() - start_time })

@app.route("/retweet/<id>", methods=["GET"])
def get_retweets(id):
    start_time = time.time()
    inCache, res = get_from_cache('retweet', id)
    if inCache:
        return json.dumps({'data': res, 'fetch_time': time.time() - start_time })
    
    res = getTweetbyID(id)
    res['retweets'] = getRetweetsByOriginalTweetID(id)
    
    add_to_cache('retweet', id, res)
    return json.dumps({'data': res, 'fetch_time': time.time() - start_time })


@app.route("/metric", methods=["GET"])
def get_metric():
    start_time = time.time()
    inCache, res = get_from_cache('metric', 'all')
    if inCache:
        return json.dumps({'data': res, 'fetch_time': time.time() - start_time })

    users = get_top_users()
    tweets = get_top_tweets()
#     users = u_temp
#     tweets = t_temp
    
    res = [users, tweets]
    
    add_to_cache('metric', 'all', res)
    return json.dumps({'data': res, 'fetch_time': time.time() - start_time })

In [ ]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
